# RAG Retrieval Optimization - Query Reformulation Supported By Amazon Bedrock Knowledge Bases

Amazon Bedrock Knowledge Bases now suppport query reformulation for complex user input queries. This feature can take a complex input query and break it into multiple sub-queries. These sub-queries will then separately go through their own retrieval steps to find relevant chunks. In this process, the subqueries having less semantic complexity might find more targeted chunks. These chunks will then be pooled and ranked together before passing them to the FM to generate a response.

Also with query reformulation natively supported in Bedrock Knowledge Base, user can take advance of this feature to imporve robustness and accuracy of their RAG solution without having to manage extrac components like third party libraries.

## Pre-req
You must run the `[workshop_setup.ipynb]`(../lab00-setup/workshop_setup.ipynb) notebook in `lab00-setup` before starting this lab.

In [ ]:
import warnings
warnings.warn("Warning: if you did not run lab00-setup, please go back and run the lab00 notebook") 

## Load the parameters

In [ ]:
print("Lab parameters....\n")
%store -r amzn10k_prefix
%store -r amzn10k_s3_path
%store -r bucket
print(amzn10k_prefix)
print(amzn10k_s3_path)
print(bucket)

print("\nload the vector db parameters....\n")
# vector parameters stored from Initial setup lab02
%store -r vector_host
%store -r vector_collection_arn
%store -r vector_collection_id
%store -r bedrock_kb_execution_role_arn

print(vector_host)
print(vector_collection_arn)
print(vector_collection_id)
print(bedrock_kb_execution_role_arn)

In [ ]:
import random
import time
import boto3
import sys

sys.path.append('../lab00-setup')
from knowledge_base import BedrockKnowledgeBase

# auth for opensearch
boto3_session = boto3.Session()
region_name = boto3_session.region_name
# try out KB using RetrieveAndGenerate API
bedrock_agent_runtime_client = boto3.client("bedrock-agent-runtime", region_name=region_name)
model_id = "us.anthropic.claude-3-7-sonnet-20250219-v1:0" # try with both claude instant as well as claude-v2. for claude v2 - "anthropic.claude-v2"

In [ ]:
suffix = random.randrange(200, 900)
kb_name = f"bedrock-query-reformation-{suffix}"
index_name = f"bedrock-query-reformulation-{suffix}"
description = "This knowledge base contain Amazon 10K financial document from 2022 and 2023"

knowledge_base = BedrockKnowledgeBase(
    kb_name=kb_name,
    kb_description=description,
    data_bucket_name=bucket,
    data_prefix=[amzn10k_prefix],
    vector_collection_arn=vector_collection_arn,
    vector_collection_id=vector_collection_id,
    vector_host=vector_host,
    bedrock_kb_execution_role_arn=bedrock_kb_execution_role_arn,
    index_name=index_name,
    suffix=suffix,
)

### Start ingestion job
Once the KB and data source is created, we can start the ingestion job.
During the ingestion job, KB will fetch the documents in the data source, pre-process it to extract text, chunk it based on the chunking size provided, create embeddings of each chunk and then write it to the vector database, in this case OSS.

In [ ]:
# ensure that the kb is available
time.sleep(30)
# sync knowledge base
knowledge_base.start_ingestion_job()

In [ ]:
kb_id = knowledge_base.get_knowledge_base_id()

### Query Reformulation
To demonstrate the functionality, we are going to more complex query contained in Aamzon's 10K financial document. This query contains a few asks that are not semantically related. When this query is embedded during the retrieval step, some aspects of the query may become diluted and therefore the relevant chunks returned may not address all components of this complex query.

To query our Knowledge Base and generate a response we will use the retrieve_and_generate API call. To use the query reformulation feature, we will include in our knowledge base configuration the additional information as shown below:

```
'orchestrationConfiguration': {
        'queryTransformationConfiguration': {
            'type': 'QUERY_DECOMPOSITION'
        }
    }

```

### > Without Query Reformulation
Let's see how the generated result looks like for the following query without using query reformulation:

"What were Amazons net sales in year 2022 and 2023? Can you compare key challenges faced by Amazon for both years?"

"What was Amazon's total net sales in 2023, and how did it compare to 2021? Also, what were the main factors contributing to the sales growth?"

"How is Amazon differentiating itself from its competition in cloud computing for years 2022 and 2023?"

In [ ]:
query = """
How is Amazon differentiating itself from its competition in cloud computing for years 2022 and 2023?
"""

In [ ]:
output_response = dict()

for x in ["DEFAULT", "QUERY_DECOMPOSITION"]:

    print(f"Start RAG generation using {x} orchestration...")
    
    knowledgeBaseConfig = {
        'knowledgeBaseId': kb_id,
        "modelArn": model_id,
        "retrievalConfiguration": {
            "vectorSearchConfiguration": {
                "numberOfResults":10
            } 
        }
    }

    if x == "QUERY_DECOMPOSITION":
        knowledgeBaseConfig["orchestrationConfiguration"] = {
                'queryTransformationConfiguration': {
                    'type': 'QUERY_DECOMPOSITION'
                }
            }
    
    response_ret = bedrock_agent_runtime_client.retrieve_and_generate(
        input={
            "text": query
        },
        retrieveAndGenerateConfiguration={
            "type": "KNOWLEDGE_BASE",
            "knowledgeBaseConfiguration": knowledgeBaseConfig
        }
    )
    
    response = response_ret['output']['text']
    
    output_response[x] = [query, response]

### > Display the results side-by-side 

Notice splitting the question into sub questions increase your chances of matching the right and complete information and generate a more comprehensive final answer.

In [ ]:
import pandas as pd
from IPython.display import display, HTML

# Create the first table
df = pd.DataFrame(output_response)

output=""
output += df.style.hide()._repr_html_()
output += "&nbsp;"

display(HTML(output))

### Clean Up

In [ ]:
# knowledge_base.delete_kb()